In [4]:
import torch
from torch import nn

# LSTM basic

In [5]:
batch_size = 64
seq_length = 10
num_directions = 1  # If the LSTM is bidirectional, num_directions should be 2, else it should be 1.
num_layers = 1

input_size = 10
hidden_size = 20  # Hidden size is number of features of the hidden state for RNN.

In [6]:
rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
input = torch.randn(batch_size, seq_length, input_size)
h0 = torch.randn(num_directions*num_layers, batch_size, hidden_size)
c0 = torch.randn(num_directions*num_layers, batch_size, hidden_size)
output, (hn, cn) = rnn(input, (h0, c0))

print(f'output: {output.size()}')
print(f'hn: {hn.size()}')
print(f'cn: {cn.size()}')

output: torch.Size([64, 10, 20])
hn: torch.Size([1, 64, 20])
cn: torch.Size([1, 64, 20])
